# تحميل المكتبات التي سوف نحتاجها ضمن هذه الوظيفة

In [1]:
import GetOldTweets3 as got
import re
from cltk.corpus.utils.importer import CorpusImporter
from cltk.corpus.arabic.utils.pyarabic import araby
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import pandas as pd
from pandas import DataFrame
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder 
from nltk.metrics import TrigramAssocMeasures

# القسم الأول : جمع البيانات وتحليلها

# أولاً  

# (1) جمع البيانات (Data Collection) -.-

# لا تنفذ هذهِ الخانة

<img src="img1.png">

In [7]:
TC = got.manager.TweetCriteria().setNear('Kingdom of Saudi Arabia')\
                                           .setSince("2019-11-16")\
                                           .setUntil("2019-11-18")\
                                           .setMaxTweets(540)

tweets = got.manager.TweetManager.getTweets(TC)
T = []
for tweet in tweets:
    T.append(tweet.text)

# لا تنفذ هذه الخانة

<img src="img2.png">

In [8]:
#التخزين ضمن ملف اكسل (CSV)
df = DataFrame(T, columns= ['tweet'])
CSV = df.to_csv (r'C:\Users\Billy Cobix 2\Desktop\nlp Home Work 1\Ali Shamma.csv',encoding="utf-8-sig",index = None,header=True)

#التخزين ضمن ملف نصي (TXT)

with open('علي شمة.txt', 'w',encoding='utf-8') as F:
    for t in T:
        F.write('%s\n' % t)

<img src="img3.png">

In [2]:
df = pd.read_csv('Ali Shamma.csv')#نقرأ الملف
df.head(10)#نظهر اول عشرة تويتات للاختبار

,tweet
0,@drmostafa64 السلام عليكم مالعمر المناسب لإخرا...
1,هذه منك تكفي
2,اللهم ارحم صدام حسين
3,والله لو علِمتُ أن فلانًا من الناسِ وقع عليه ظ...
4,يا اخت نوال تغريدتج هذي فيها استدعاء للنيابة و...
5,على ذمة الراوي يقولون كان استاذ بالجامعة ودائم...
6,حطي أفتاري هذاpic.twitter.com/NgtlV2TEtV
7,تلاوته جميلةhttps://twitter.com/BrkaQ8/status/...
8,انا آسف على كل دقيقة مرّت بدونك
9,هلااا


<img src="img4.png">

In [3]:
#تخزين التويتات ضمن tuble
data = [tuple(x) for x in df.values]

#اخذ اول عمود فقط والذي يمثل التويتات ونخزنها ضمن سلسلة من اجل اجراء المعالجة عليها
data_list = []
for i in range(540):
    d = data[i][0]
    data_list.append(d)    
data_list

['@drmostafa64 السلام عليكم مالعمر المناسب لإخراج الطفل من غرفة الأم والأب بحيث لا يؤثر عليه سلبياً من النواحي العاطفية والاجتماعية والنفسية ؟',
 'هذه منك تكفي',
 'اللهم ارحم صدام حسين',
 'والله لو علِمتُ أن فلانًا من الناسِ وقع عليه ظُلم، وهذا الإنسان أعلم عِلم اليقين ببغضيه إياي؛ لرجوت الله أن يرفع الظلم عنه. فكيف إذا كان أخاً لي في الدِّين؟ سيقول مَن توشّح قلبه الحِقد: ”إنها مثالية كاذِبة”، بل هي يا أخي أُسس أخلاقية، ومبادئ قال عنها دنقل: ”هي أشياء لا تُشترى”.',
 'يا اخت نوال تغريدتج هذي فيها استدعاء للنيابة وهالأشكال هذي ماعندها مانع تسحبچ للنيابة اني ناصح لچ',
 'على ذمة الراوي يقولون كان استاذ بالجامعة ودائما يخش وجه تحت النظارات الشمسية اللي ماعمره نزلها لدرجة الطلاب شكوا ان ينام فيهم وكان انطوائي كله بالمكتب',
 'حطي أفتاري هذاpic.twitter.com/NgtlV2TEtV',
 'تلاوته جميلةhttps://twitter.com/BrkaQ8/status/1193296952873631749 …',
 'انا آسف على كل دقيقة مرّت بدونك',
 'هلااا',
 'ارجع يازمان ... #ساحه_الارادهhttps://twitter.com/abdullahrsm/status/1196181012377677824 …',
 'نقطة',
 'حُب ل

# (2) تنظيف النص والتطبيع (Text Cleaning and Normalization) -.-

# إزالة الروابط أو الصور

<img src="img5.png">

In [4]:
#النمط المعبر عن الصور الموجودة ضمن التغريدات
pattern1 = r"pic.twitter.com/(.)*"

#النمط المعبر عن الروابط الموجود ضمن التغريدات
pattern2 = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"

#اضافي : النمط المعبر عن المنشن لمستخدم معين 
patternM = r"@[A-Za-z0-9]*"

#انشاء سلسلة جديدة من اجل تخزين التغريدات الجديدة فيها بعد ازالة الروابط والصور
data_d_urls_pic = []
for i in range(540):
    text = re.sub(pattern1, "", str(data_list[i]))#ازالة الصور من التغريدة
    text = re.sub(pattern2,"", text)#ازالة الروابط من التغريدة
    text = re.sub(patternM,"", text)#ازالة المنشن من التغريدة
    data_d_urls_pic.append(text)#اضافة التغريدة الجديدة الى السلسلة الجديدة

#طباعة التغريدات بعد المعالجة
data_d_urls_pic

[' السلام عليكم مالعمر المناسب لإخراج الطفل من غرفة الأم والأب بحيث لا يؤثر عليه سلبياً من النواحي العاطفية والاجتماعية والنفسية ؟',
 'هذه منك تكفي',
 'اللهم ارحم صدام حسين',
 'والله لو علِمتُ أن فلانًا من الناسِ وقع عليه ظُلم، وهذا الإنسان أعلم عِلم اليقين ببغضيه إياي؛ لرجوت الله أن يرفع الظلم عنه. فكيف إذا كان أخاً لي في الدِّين؟ سيقول مَن توشّح قلبه الحِقد: ”إنها مثالية كاذِبة”، بل هي يا أخي أُسس أخلاقية، ومبادئ قال عنها دنقل: ”هي أشياء لا تُشترى”.',
 'يا اخت نوال تغريدتج هذي فيها استدعاء للنيابة وهالأشكال هذي ماعندها مانع تسحبچ للنيابة اني ناصح لچ',
 'على ذمة الراوي يقولون كان استاذ بالجامعة ودائما يخش وجه تحت النظارات الشمسية اللي ماعمره نزلها لدرجة الطلاب شكوا ان ينام فيهم وكان انطوائي كله بالمكتب',
 'حطي أفتاري هذا',
 'تلاوته جميلة…',
 'انا آسف على كل دقيقة مرّت بدونك',
 'هلااا',
 'ارجع يازمان ... #ساحه_الاراده…',
 'نقطة',
 'حُب لأول مُحادثة ، لأول نقاش ، ولأول أُغنية أهديتني أياها ، لفجرنا الأول ، وابتسامتي الأولى ، لأول شعُور أحسست به بقربك ، لأول حديث دار بيننا ، لأول ضحكه ضح

# text normalization(توحيد شكل النص)

<img src="img6.png">

In [5]:
pattern3 = r"ة"
pattern4 = r"ء|أ|إ|آ"
pattern5 = r"ئ"
pattern6 = r"ؤ"
pattern7 = r"(.)\1{2,}"

#انشاء سلسلة جديدة من أجل تخزين فيها التغريدات بعد توحيد شكل النص فيها
data_normalization = []

for i in range(540):
    text = re.sub(pattern3,"ه",data_d_urls_pic[i])#استبدال التاء المربوطة بهاء
    text = re.sub(pattern4,"ا",text)#استبدال الهمزة والمدة بالف
    text = re.sub(pattern5,"ى",text)#استبدال النبرة بالف مقصورة
    text = re.sub(pattern6,"و",text)#استبدال الهمزة على واو بواو
    text = araby.strip_tashkeel(text)#ازالة جميع التشكيلات مع الشدة
    text.strip()#ازالة المسافات من الاطراف
    text = re.sub(pattern7,r"\1\1", text)#ازالة الاحرف المتكررة
    data_normalization.append(text)#اضافة التغريدة الجديدة الى السلسلة الجديدة

#طباعة التغريدات بعد المعالجة
data_normalization

[' السلام عليكم مالعمر المناسب لاخراج الطفل من غرفه الام والاب بحيث لا يوثر عليه سلبيا من النواحي العاطفيه والاجتماعيه والنفسيه ؟',
 'هذه منك تكفي',
 'اللهم ارحم صدام حسين',
 'والله لو علمت ان فلانا من الناس وقع عليه ظلم، وهذا الانسان اعلم علم اليقين ببغضيه اياي؛ لرجوت الله ان يرفع الظلم عنه. فكيف اذا كان اخا لي في الدين؟ سيقول من توشح قلبه الحقد: ”انها مثاليه كاذبه”، بل هي يا اخي اسس اخلاقيه، ومبادى قال عنها دنقل: ”هي اشياا لا تشترى”.',
 'يا اخت نوال تغريدتج هذي فيها استدعاا للنيابه وهالاشكال هذي ماعندها مانع تسحبچ للنيابه اني ناصح لچ',
 'على ذمه الراوي يقولون كان استاذ بالجامعه وداىما يخش وجه تحت النظارات الشمسيه اللي ماعمره نزلها لدرجه الطلاب شكوا ان ينام فيهم وكان انطواىي كله بالمكتب',
 'حطي افتاري هذا',
 'تلاوته جميله…',
 'انا اسف على كل دقيقه مرت بدونك',
 'هلاا',
 'ارجع يازمان .. #ساحه_الاراده…',
 'نقطه',
 'حب لاول محادثه ، لاول نقاش ، ولاول اغنيه اهديتني اياها ، لفجرنا الاول ، وابتسامتي الاولى ، لاول شعور احسست به بقربك ، لاول حديث دار بيننا ، لاول ضحكه ضحكناها سويا',
 'قومو يان

# (3)Text Analysis (تحليل النص)

<img src="img7.png">

In [6]:
#جمع جميع التغريدات في نص واحد فقط
AllTexts = ""
for text in data_normalization:
    AllTexts = AllTexts + text

#تقسيم النص المجموع الى كلمات    
tokenizer = nltk.tokenize.WhitespaceTokenizer()
tokens = tokenizer.tokenize(AllTexts)

# مجموعة الكلمات الاكثر شيوعاً

<img src="img8.png">

# 1 - unigram

In [7]:
#حساب التوزيع التكراري للكلمات ضمن النص
fdist = nltk.FreqDist(w for w in tokens)

#طباعة اكثر الكلمات شيوعاً
print("The Most Common Words : ")
print(fdist.most_common(32))

#طباعة الكلمات الاقل وروداً
print("The hapaxes Words : ")
print(fdist.hapaxes()[:10])

The Most Common Words : 
[('من', 92), ('في', 81), ('الله', 43), ('على', 42), ('ان', 38), ('و', 28), ('لا', 27), ('اللي', 27), ('ما', 25), ('ولا', 23), ('،', 22), ('يا', 21), ('كل', 20), ('..', 15), ('او', 15), ('اذا', 14), ('كان', 14), ('الى', 14), ('لك', 14), ('عن', 14), ('كيف', 13), ('مع', 13), ('والله', 13), ('حتى', 13), ('بس', 13), ('يوم', 13), ('فيه', 12), ('شي', 12), ('شيا', 12), ('هو', 11), ('بعد', 11), ('هذا', 11)]
The hapaxes Words : 
['السلام', 'مالعمر', 'المناسب', 'لاخراج', 'الطفل', 'غرفه', 'الام', 'والاب', 'بحيث', 'يوثر']


# 2 - bigram

<img src="img9.png">

In [8]:
#اخذ كل كلمتين متتاليتين مع بعضهما البعض
bigrams = nltk.bigrams(tokens)

#حساب التوزيع التكراري لثنائيات الكلمات ضمن النص
fdist1 = nltk.FreqDist(bi for bi in bigrams)

#طباعة اكثر ثنائيات الكلمات شيوعاً
print("The Most Common bigrams : ")
print(fdist1.most_common(10))

#طباعة الثنائيات الاقل وروداً
print("The hapaxes bigrams : ")
print(fdist1.hapaxes()[:10])

The Most Common bigrams : 
[(('شاا', 'الله'), 5), (('،', 'لاول'), 4), (('ان', 'شاا'), 4), (('الله', 'ان'), 3), (('بن', 'سلمان'), 3), (('وادي', 'الرمه'), 3), (('شي', 'غلط'), 3), (('،', 'كل'), 3), (('طوبى', 'للقابضين'), 3), (('للقابضين', 'على'), 3)]
The hapaxes bigrams : 
[('السلام', 'عليكم'), ('عليكم', 'مالعمر'), ('مالعمر', 'المناسب'), ('المناسب', 'لاخراج'), ('لاخراج', 'الطفل'), ('الطفل', 'من'), ('من', 'غرفه'), ('غرفه', 'الام'), ('الام', 'والاب'), ('والاب', 'بحيث')]


# 3 - trigram

<img src="img10.png">

In [9]:
#اخذ كل ثلاثية من الكلمات المتتالية مع بعضها البعض
Tg = ngrams(tokens, 3)

#حساب التوزيع التكراري لثلاثيات الكلمات ضمن النص
fdist2 = nltk.FreqDist(tr for tr in Tg)

#طباعة اكثر الثلاثيات شيوعاً
print("The Most Common trigrams : ")
print(fdist2.most_common(10))

#طباعة الثلاثيات الاقل وروداً
print("The hapaxes trigrams : ")
print(fdist2.hapaxes()[:10])

The Most Common trigrams : 
[(('ان', 'شاا', 'الله'), 4), (('طوبى', 'للقابضين', 'على'), 3), (('للقابضين', 'على', 'الجمر'), 3), (('في', 'السيطره', 'على'), 2), (('ابني', 'لك', 'سمعه'), 2), (('توفر', 'معلومه', 'سيتم'), 2), (('هم', 'جزا', 'لا'), 2), (('جزا', 'لا', 'يتجزا'), 2), (('لا', 'يتجزا', 'من'), 2), (('التي', 'ليس', 'لها'), 2)]
The hapaxes trigrams : 
[('السلام', 'عليكم', 'مالعمر'), ('عليكم', 'مالعمر', 'المناسب'), ('مالعمر', 'المناسب', 'لاخراج'), ('المناسب', 'لاخراج', 'الطفل'), ('لاخراج', 'الطفل', 'من'), ('الطفل', 'من', 'غرفه'), ('من', 'غرفه', 'الام'), ('غرفه', 'الام', 'والاب'), ('الام', 'والاب', 'بحيث'), ('والاب', 'بحيث', 'لا')]


<img src="img11.png">

# bigram collocations

In [10]:
#الحصول على تجميعات ثنائيات الكلمات من النص
Cbigram = BigramCollocationFinder.from_words(tokens)

#اخذ افضل 32 تجميع
Cbigram.nbest(BigramAssocMeasures.likelihood_ratio, 32)

[('طوبى', 'للقابضين'),
 ('وادي', 'الرمه'),
 ('شاا', 'الله'),
 ('بن', 'سلمان'),
 ('،', 'لاول'),
 ('هذه', 'الايه'),
 ('شي', 'غلط'),
 ('ادى', 'لتطاير'),
 ('اذ', 'نعاين'),
 ('اسلافنا', 'فهي'),
 ('اقسم', 'بالله'),
 ('الخزي', 'ونسمعه'),
 ('الزنادقه', 'بدات'),
 ('السماا.ا.هفلا', 'تذهب'),
 ('الشباب', 'وتسرعهم'),
 ('العموميه', 'البريطانيه'),
 ('العهر', 'والمجون'),
 ('الغير', 'يحذرون'),
 ('المكتبه', 'العموميه'),
 ('المنظر', 'يخبرك'),
 ('المهج', 'فما'),
 ('الهدم', 'واضحه'),
 ('باذنيهم', 'وقرا'),
 ('بالتراكمات', 'الاخلاقيه'),
 ('بدات', 'معالم'),
 ('بعنان', 'فرسه،'),
 ('بنفسك', 'حسرات'),
 ('تبثه', 'قناه'),
 ('تثير', 'حفيظه'),
 ('تذهب', 'بنفسك'),
 ('تعالج', 'علاج'),
 ('توفر', 'معلومه')]

<img src="img12.png">

# trigram collocations

In [11]:
#الحصول على تجميعات ثلاثيات الكلمات من النص
Ctrigram = TrigramCollocationFinder.from_words(tokens)

#اخذ افضل 32 تجميع
Ctrigram.nbest(TrigramAssocMeasures.likelihood_ratio, 32)

[('ان', 'شاا', 'الله'),
 ('طوبى', 'للقابضين', 'على'),
 ('للقابضين', 'على', 'الجمر'),
 ('الجمر', 'طوبى', 'للقابضين'),
 ('ادى', 'لتطاير', 'المهج'),
 ('اسلافنا', 'فهي', 'اوضح'),
 ('الزنادقه', 'بدات', 'معالم'),
 ('السماا.ا.هفلا', 'تذهب', 'بنفسك'),
 ('الشباب', 'وتسرعهم', 'ولربما'),
 ('العهر', 'والمجون', '(الام'),
 ('المكتبه', 'العموميه', 'البريطانيه'),
 ('المنظر', 'يخبرك', 'بالتراكمات'),
 ('المهج', 'فما', 'قيمتنا'),
 ('الهدم', 'واضحه', 'للعيان'),
 ('بدات', 'معالم', 'الهدم'),
 ('بنفسك', 'حسرات', 'فهذا'),
 ('تبثه', 'قناه', 'العهر'),
 ('تثير', 'حفيظه', 'المسلم'),
 ('تذهب', 'بنفسك', 'حسرات'),
 ('تعالج', 'علاج', 'جذري'),
 ('توفر', 'معلومه', 'سيتم'),
 ('طريق', 'الزنادقه', 'بدات'),
 ('فالقد', 'نخرت', 'بقيم'),
 ('فمجرد', 'نظره', 'ثاقبه'),
 ('قناه', 'العهر', 'والمجون'),
 ('قواميس', 'المكتبه', 'العموميه'),
 ('كانما', 'باذنيهم', 'وقرا'),
 ('كانوا', 'كانما', 'باذنيهم'),
 ('كبد', 'السماا.ا.هفلا', 'تذهب'),
 ('كثره', 'قضايا', 'الاعراض'),
 ('لتطاير', 'المهج', 'فما'),
 ('للعلماا', 'كانوا', 'كانما')]

# الاخطاء الاملائية

<img src="img121.png">

In [22]:
#قراءة ملف كلمات اللغة الاكثر شيوعاً
with open('AC.txt', 'r',encoding='utf-8') as F:
    cor = F.read()
    
list_of_wrong_words = []#انشاء سلسلة الخطاء الاملائية

#تقطيع النص الى كلمات بالاعتماد على space
tokenizer = nltk.tokenize.WhitespaceTokenizer()
ts = tokenizer.tokenize(cor)
yes = 0
for token in tokens:#من اجل كل كلمة من التغريدات
    yes = 0
    for t in ts:#من اجل كل كلمة من كلمات اللغة العربية
        if token == t:#اذا كانت الكلمة صحيحة
            yes = yes+1#اضف الى قيمة المتحول 1
    if yes == 0:#دليل على عدم ورود هذه الكلمة ضمن ملف كلمات اللغة العربية
        list_of_wrong_words.append(token)#اضافة هذه الكلمة الى سلسلة الاخطاء الاملائية
            
print(list_of_wrong_words)

['السلام', 'عليكم', 'مالعمر', 'المناسب', 'لاخراج', 'الطفل', 'يوثر', 'العاطفيه', 'والاجتماعيه', 'والنفسيه', '؟هذه', 'تكفياللهم', 'صدام', 'حسينوالله', 'فلانا', 'ظلم،', 'ببغضيه', 'اياي؛', 'لرجوت', 'الظلم', 'عنه.', 'الدين؟', 'توشح', 'الحقد:', '”انها', 'كاذبه”،', 'اسس', 'اخلاقيه،', 'ومبادى', 'دنقل:', '”هي', 'اشياا', 'تشترى”.يا', 'نوال', 'تغريدتج', 'هذي', 'استدعاا', 'للنيابه', 'وهالاشكال', 'هذي', 'ماعندها', 'مانع', 'تسحبچ', 'للنيابه', 'اني', 'ناصح', 'لچعلى', 'ذمه', 'الراوي', 'بالجامعه', 'وداىما', 'يخش', 'الشمسيه', 'اللي', 'ماعمره', 'نزلها', 'لدرجه', 'شكوا', 'وكان', 'انطواىي', 'كله', 'بالمكتبحطي', 'افتاري', 'هذاتلاوته', 'جميله…انا', 'دقيقه', 'مرت', 'بدونكهلااارجع', 'يازمان', '..', '#ساحه_الاراده…نقطهحب', 'لاول', 'محادثه', '،', 'لاول', '،', 'ولاول', 'اغنيه', 'اهديتني', 'اياها', '،', 'لفجرنا', 'الاول', '،', 'وابتسامتي', 'الاولى', '،', 'لاول', 'احسست', 'به', '،', 'لاول', 'دار', '،', 'لاول', 'ضحكناها', 'سوياقومو', 'يانكبه', 'جاا', 'مطركممطرنا', 'ورحمته', 'يبطي', 'ياشتاا', 'الدمام', 'للحين', 'جو',

# (4) (Trending Topics) المواضيع الشائعة

<img src="img13.png">

In [15]:
#تقسيم النص بالاعتماد على النقطة
tokenizer = nltk.tokenize.WordPunctTokenizer()
tokens = tokenizer.tokenize(AllTexts)

#اخذ كل كلمتين متتاليتين مع بعضهما البعض
bigrams = nltk.bigrams(tokens)

#حساب التوزيع التكراري المشروط
CFDbigram = nltk.ConditionalFreqDist(bigrams)

#اظهار الهاشتاغات الاكثر شيوعاً
CFDbigram['#']

FreqDist({'العسكري_يشتكي_سلمان_الفرج': 3, 'طهران': 3, 'ساحه_الاراده': 2, 'ارامكو_السعوديه': 1, 'المولد_النبوي': 1, 'المساعده_المقطوعه': 1, 'العيد_الوطني_49_المجيد': 1, 'مجلس_الامه': 1, 'حقيقه_يجب_ادراكها': 1, 'الكويتحلم': 1, ...})

# النهاية -.-

# تقديم الطالب : علي منذر شمة

# الفئة : الثانية